In [1]:
# Assumptions:
# dimensions image W 2x user image W;
# H is the same for both;
# user image has fixed W and H

# Get the MRP from the user image - left foot in this case
# On the user image, D will be MRP.W;
# On the dimensions image, crop LEFT SIDE by D from the MRP of the dim image
#     dim.MRP.W = 300 _____ (300-d:user.W-300)
# On dimensions image crop RightSide by user.W
#     (0:1500) - original dimemsnions width
#     (300:1500) - left side crop
#     (300:300+userW) - right side crop with reference to left side crop

In [2]:
# Create a dictionary with these
body_parts={
    0:"nose",
    1:"left_eye_inner",
    2:"left_eye",
    3:"left_eve_outer",
    4:"right_eye_inner",
    5:"right_eye",
    6:"right_eye_outer",
    7:"left_ear",
    8:"right_ear",
    9:"mouth_left",
    10:"mouth_right",
    11:"left_shoulder",
    12:"right_shoulder",
    13:"left_elbow",
    14:"right_elbow",
    15:"left_wrist",
    16:"right_wrist",
    17:"left_pinky",
    18:"right_pinky",
    19:"left index",
    20:"right_index",
    21:"left_thumb",
    22:"right_thumb",
    23:"left_hip",
    24:"right_hip",
    25:"left_knee",
    26:"right_knee",
    27:"left_ankle",
    28:"right_ankle",
    29:"left_heel",
    30:"right_heel",
    31:"left_foot_index",
    32:"right_foot_index"
    }

# ref_body_dict=[]
# for i in body_parts:
#     x=body_parts[i]
#     y=i
#     ref_body_dict.append((x,y))
# ref_body_dict=dict(ref_body_dict)
# ref_body_dict.keys()

In [83]:
import mediapipe as mp
import cv2
import time
import numpy as np

img=cv2.imread(r'images\tree.jpg')
def med_pipe(img):
    mpPose = mp.solutions.pose
    pose = mpPose.Pose()
    mpDraw = mp.solutions.drawing_utils # For drawing keypoints
    points = mpPose.PoseLandmark # Landmarks

    # img = cv2.imread(r'images\tree.jpg')
    imageWidth, imageHeight = img.shape[:2]
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    blank = np.zeros(img.shape) # Blank image
    results = pose.process(imgRGB)

    if results.pose_landmarks:
        mpDraw.draw_landmarks(blank, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blank
        landmarks = results.pose_landmarks.landmark

    return results

med_pipe(img)
# cv2.imshow("Image", img)
# cv2.imshow("blank",blank)
# # cv2.imwrite('Images_output\yogframe_points.jpg',blank)
# cv2.waitKey()
# cv2.destroyAllWindows()

mediapipe.python.solution_base.SolutionOutputs

In [91]:
comb_list=[]
for id, lm in enumerate(results.pose_landmarks.landmark):
    h, w,c = img.shape
    # print(id, lm)
    cx, cy = int(lm.x*w), int(lm.y*h)
    comb_list.append((cx,cy))

comb_list=dict(enumerate(comb_list))


In [92]:
body_points=[]
for i in comb_list:
    x=body_parts[i]
    y=comb_list[i]
    body_points.append((x,y))

body_points=dict(body_points)

In [93]:
body_points.keys()

dict_keys(['nose', 'left_eye_inner', 'left_eye', 'left_eve_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'])

In [94]:
req_body_points=["left_ankle","right_ankle","nose","left_wrist","right_wrist","left_hip","right_hip","right_knee"]
mrp_user=[]
for i in body_points:
    if i in req_body_points:
        x=i
        y=body_points[i]
        mrp_user.append((x,y))
mrp_user=dict(mrp_user)
mrp_user


{'nose': (444, 161),
 'left_wrist': (450, 74),
 'right_wrist': (434, 76),
 'left_hip': (458, 347),
 'right_hip': (408, 343),
 'right_knee': (328, 402),
 'left_ankle': (435, 537),
 'right_ankle': (420, 383)}

In [95]:
import cv2

user_img=cv2.imread(r"images\tree.jpg")
dim_img=cv2.imread("images\dim_tree_org.png")

print(user_img.shape)
print(dim_img.shape)

(600, 800, 3)
(615, 314, 3)


In [96]:
#Basic Idea 
dim_img=np.zeros((100,300))
user_img=np.zeros((100,150))

dim_mrp=(50,100)
user_mrp=(30,100)

dim_crop=dim_mrp[0]-user_mrp[0]
dim_img=dim_img[:,dim_crop:dim_crop+(user_img.shape[1])]

print(dim_img.shape)
print(user_img.shape)

(100, 150)
(100, 150)


In [150]:
Poses={'triangle':('right_ankle',80),'wheel':('left_ankle',100),'tree':('left_wrist','left_ankle','left_ankle',125,'left_ankle')}


In [151]:
#Autocrop
import cv2

user_img=cv2.imread(r"images\tree.jpg")
D=Poses['tree'][3]
cropht1=mrp_user[Poses['tree'][0]][1]-70
cropht2=mrp_user[Poses['tree'][1]][1]+30
cropW1=mrp_user[Poses['tree'][2]][0]-D
cropW2=mrp_user[Poses['tree'][4]][0]+100
user_img=user_img[cropht1:cropht2,cropW1:cropW2]
print(user_img.shape)
print(f'distance to left leg is {D}')
cv2.imshow("Image",user_img)
cv2.waitKey()
cv2.destroyAllWindows()


(563, 225, 3)
distance to left leg is 125


In [152]:
# Things Done:
    # Get the Sample Image Cropped from the Position of Body Parts and set distance of leg from left 
# Things to do:
    # Need to somehow manage similare shape and d for Sample and Dimensions Image

In [153]:
import cv2
dim_img=cv2.imread('images\dim_tree_org.png')
dim_img=dim_img[cropht1:,:cropW2]
print(dim_img.shape)
# cv2.imshow("Image",dim_img)
# cv2.waitKey()
# cv2.destroyAllWindows()

(611, 314, 3)


In [154]:
import cv2
import numpy as np

img1=dim_img
img2=user_img

# img2 = img2[0:480,0:425]
# img1= cv2.resize(img1,img2.shape[1::-1])
img2= cv2.resize(img2,img1.shape[1::-1])
added_image = cv2.addWeighted(img1,0.5,img2,0.4,0)


cv2.imshow('Blended Image',added_image)
cv2.imwrite("Trial&error for correct overlay\Automated_output_tree.jpg",added_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [155]:
print(user_img.shape[1]//user_img.shape[1])
print(dim_img.shape[1]//dim_img.shape[0])



1
0


In [156]:
import cv2
dim_img=cv2.imread('images\dim_image_org.png')
